In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf

from IPython.display import Audio as ipy_audio
from quicktranscribe.wave import read_audio_section
from quicktranscribe import kde

In [ ]:
from mogra.datatypes import ratio_to_sswar

In [ ]:
gps_file = "GhodePeSawaar.mp3"
ngtj_file = "NasatesaGhariTuJevha.wav"
bhoop_file = "Omkar Dadarkar - Raag Bhoopali.mp3"
malkauns_file = "Ajoy Chakrabarty - Malkauns.mp3"
rjha_sarang = "rjha_sarang.mp3"
audios_dir = "misc-audios/"

## quicktranscribe for Sarang-Megh Re ma comparison

sarang 4:15 to 4:58 <br>
megh 5:23 to 5:41, 5:55 to 6:25 <br>
megh best 7:10 to 8:42 <br>
comparison 8:48 to 9:21 <br>

### comparison chunk

In [ ]:
# comparison
start = 8*60+48
end = 9*60+21
y_stereo, sr = read_audio_section(audios_dir + rjha_sarang, start, end)
y = librosa.to_mono(y_stereo.T)
print(y.shape)

In [ ]:
ipy_audio(data=y, rate=sr)

In [ ]:
f0, _, voiced_probab = librosa.pyin(y, fmin=50, fmax=200)

Find tonic

In [ ]:
gtonic = 60  # TEMP GUESS

kde_sample = kde.extract(y, sr=sr, tonic=gtonic)
peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
print(peaks)

plt.figure(figsize=(5,3))
plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
plt.xlabel("relative note index")
plt.ylabel("normalized duration")

In [ ]:
print(np.array(peaks) * 12/len(kde_sample))

In [ ]:
sa = 0.05
ctonic = librosa.midi_to_hz(librosa.hz_to_midi(gtonic) + sa)
print(ctonic)

In [ ]:
ctonic = 60.17353727216203
sarang_R = ctonic*9/8
megh_R = ctonic*10/9
sarang_n = ctonic*9/10
megh_n = ctonic*8/9
both_P = ctonic*3/2
both_m = ctonic*4/3

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f0, linewidth=1)
plt.rcParams["figure.dpi"] = 500
plt.yticks([megh_n, sarang_n, ctonic, megh_R, sarang_R, both_m, both_P])
plt.ylim([52,97])
plt.grid(True)
plt.title("sarang-megh comparison chunk")
plt.show()

### another chunk

In [ ]:
start = 4*60+15
end = 4*60+58
y_stereo, sr = read_audio_section(audios_dir + rjha_sarang, start, end)
y = librosa.to_mono(y_stereo.T)
print(y.shape)

In [ ]:
f0, _, voiced_probab = librosa.pyin(y, fmin=50, fmax=200)

In [ ]:
gtonic = 60  # TEMP GUESS

kde_sample = kde.extract(y, sr=sr, tonic=gtonic)
peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
# print(peaks)

plt.figure(figsize=(5,3))
plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
plt.xlabel("relative note index")
plt.ylabel("normalized duration")
print(np.array(peaks) * 12/len(kde_sample))

In [ ]:
sa = 0.0
ctonic = librosa.midi_to_hz(librosa.hz_to_midi(gtonic) + sa)
print(ctonic)

In [ ]:
sarang_R = ctonic*9/8
megh_R = ctonic*10/9
sarang_n = ctonic*9/10
megh_n = ctonic*8/9
both_P = ctonic*3/2
both_m = ctonic*4/3
plt.figure(figsize=(10,4))
plt.plot(f0, linewidth=1)
plt.rcParams["figure.dpi"] = 500
plt.yticks([megh_n, sarang_n, ctonic, megh_R, sarang_R, both_m, both_P])
plt.ylim([52,97])
plt.grid(True)
plt.show()

In [ ]:
ipy_audio(data=y, rate=sr)

### yet another chunk

In [ ]:
start = 7*60+15
end = 7*60+42
y_stereo, sr = read_audio_section(audios_dir + rjha_sarang, start, end)
y = librosa.to_mono(y_stereo.T)
print(y.shape)

In [ ]:
f0, _, voiced_probab = librosa.pyin(y, fmin=50, fmax=200)

In [ ]:
gtonic = 60  # TEMP GUESS

kde_sample = kde.extract(y, sr=sr, tonic=gtonic)
peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
# print(peaks)

plt.figure(figsize=(5,3))
plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
plt.xlabel("relative note index")
plt.ylabel("normalized duration")
print(np.array(peaks) * 12/len(kde_sample))

In [ ]:
sa = 0
ctonic = librosa.midi_to_hz(librosa.hz_to_midi(gtonic) + sa)
print(ctonic)

In [ ]:
sarang_R = ctonic*9/8
megh_R = ctonic*10/9
sarang_n = ctonic*9/10
megh_n = ctonic*8/9
both_P = ctonic*3/2
both_m = ctonic*4/3
sarang_nh = 2*ctonic*9/10
megh_nh = 2*ctonic*8/9
plt.figure(figsize=(10,4))
plt.plot(f0, linewidth=1)
plt.rcParams["figure.dpi"] = 500
plt.yticks([megh_n, sarang_n, ctonic, megh_R, sarang_R, both_m, both_P, megh_nh, sarang_nh, ctonic*2])
plt.ylim([47,137])
plt.grid(True)
plt.show()

In [ ]:
ipy_audio(data=y, rate=sr)

## quicktranscribe for bhoop --> get notes

In [ ]:
start = 16*60+10
end = 16*60+36
y_stereo, sr = read_audio_section(audios_dir + bhoop_file, start, end)
y = librosa.to_mono(y_stereo.T)
print(y.shape)

In [ ]:
ipy_audio(data=y, rate=sr)

In [ ]:
f0, _, voiced_probab = librosa.pyin(y, fmin=50, fmax=200)

In [ ]:
gtonic = 80  # TEMP GUESS

kde_sample = kde.extract(y, sr=sr, tonic=gtonic)
peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
# print(peaks)

plt.figure(figsize=(5,3))
plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
plt.xlabel("relative note index")
plt.ylabel("normalized duration")
print(np.array(peaks) * 12/len(kde_sample))

In [ ]:
sa = 11.5
ctonic = librosa.midi_to_hz(librosa.hz_to_midi(gtonic) + sa)
print(ctonic)

In [ ]:
# generate a waveform of the tonic to verify
t = np.linspace(0, 2, sr)
y_tonic = 0.5 * np.sin(2 * np.pi * ctonic * t)
ipy_audio(data=y_tonic, rate=sr)

In [ ]:
BHOOP_SWARS = np.array([
    5/6, 1, 10/9, 5/4, 3/2, 5/3, 2
])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(f0, linewidth=1)
plt.rcParams["figure.dpi"] = 500
plt.grid(True)
plt.yticks(BHOOP_SWARS / 2 * ctonic)
plt.title("bhoopali")
plt.show()

In [ ]:
ipy_audio(data=y, rate=sr)

## Raag-Conditioned Transcription

Given a pitch contour `f0`, a set of "raag notes" `raag0` that span the range of f0, and a tolerance `tol` in Hz

Return a list of notes based on entering and exiting the "band" of `[r0-tol, r0+tol]` for each note `r0` in the raag

In [ ]:
raag0 = BHOOP_SWARS / 2 * ctonic
tol = 2.0

In [ ]:
def raag_transcribe(raag0: np.ndarray, tol: float, f0: np.ndarray) -> list[int]:
    """
    return a list of indices of the raag notes as they are traversed in f0
    TODO(neeraja): add a time component
    """
    notes = []
    curr_note = -1
    for ff in f0:
        if ff == 0:
            continue
        # find the closest note
        dists = np.abs(raag0 - ff)
        min_dist = np.min(dists)
        if min_dist < tol:
            new_note = np.argmin(dists)
            if new_note != curr_note:
                notes.append(new_note)
                curr_note = new_note
    return notes

In [ ]:
# transcription = [
#     ratio_to_sswar(BHOOP_SWARS[note])
#     for note in raag_transcribe(raag0, tol, f0)
# ]

In [ ]:
TEMP_BHOOP_MAP = {0: ",D", 1: "S", 2: "R", 3: "G", 4: "P", 5: "D", 6: "`S"}
transcription = [
    TEMP_BHOOP_MAP[note]
    for note in raag_transcribe(raag0, tol, f0)
]

In [ ]:
" ".join(transcription)

## Librosa spectral features

In [ ]:
start = 6
end = 16
y_stereo, sr = read_audio_section(malkauns_file, start, end)
y = librosa.to_mono(y_stereo.T)

In [ ]:
ipy_audio(data=y, rate=sr)

STFT

In [ ]:
S = np.abs(librosa.stft(y, n_fft=4096))**2
img = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')

In [ ]:
S = np.abs(librosa.stft(y, n_fft=4096))**2
chroma = librosa.feature.chroma_stft(S=S, sr=sr)
img = librosa.display.specshow(chroma, y_axis='chroma_h', x_axis='time', thaat='bhairavi', Sa=1)

CQT

In [ ]:
C = np.abs(librosa.cqt(y, sr=sr))
img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')

In [ ]:
chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
img = librosa.display.specshow(chroma_cq, y_axis='chroma_h', x_axis='time', thaat='bhairavi', Sa=1)

VQT

In [ ]:
V = np.abs(librosa.vqt(y, sr=sr, bins_per_octave=36, fmin=librosa.note_to_hz('C2')))
img = librosa.display.specshow(librosa.amplitude_to_db(V, ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')

In [ ]:
chroma_vq = librosa.vqt(y, sr=sr, bins_per_octave=36, fmin=librosa.note_to_hz('C2'))
img = librosa.display.specshow(chroma_vq, y_axis='vqt_fjs', x_axis='time', thaat='bhairavi', Sa=1, intervals='ji5')

Misc

In [ ]:
""" Zero crossing:
The  higher this is, the less reliable our computed frequency is?? Why??
"""
zc = librosa.feature.zero_crossing_rate(y)[0]
plt.plot(zc)

In [ ]:
""" RMS:
Correlates with the energy in the signal over time. The RMS is probably over the f domain, so a sharp f --> a higher RMS??
"""
rms = librosa.feature.rms(y=y)[0]
plt.plot(rms)

In [ ]:
mfcc = librosa.feature.mfcc(y=y, sr=sr, dct_type=2)
plt.figure(figsize=(10,5))
# ax = plt.gca()
img = librosa.display.specshow(mfcc, x_axis='time')
# ax.matshow(mfcc)

## speech_recongition example

In [ ]:
import speech_recognition as sr

In [ ]:
r = sr.Recognizer()

# Reading Audio File and storing in a variable
with sr.AudioFile(ngtj_file) as source:
    audio_text = r.listen(source)

In [ ]:
text = r.recognize_google(audio_text, language="mr")